In [1]:
import random
import numpy as np
import pandas as pd
# Visualisation imports
import matplotlib.pyplot as plt
# Scikit learn for preprocessing
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
# Keras Imports - CNN
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Conv1D, MaxPool2D, Flatten, Dropout, Convolution2D, Activation
from keras.optimizers import Adam, SGD
from keras.utils.np_utils import to_categorical
import pandas as pd
from sklearn.linear_model import LogisticRegression
from __future__ import division

Using TensorFlow backend.


In [2]:
data_path = "/Users/MeryemMhamdi/Desktop/Lauzhack_creditSuisse-master/data/"

In [3]:
train = pd.read_csv(data_path+'train.csv', header=0, sep=",", index_col=0, parse_dates=True)

In [14]:
print(list(train.columns))

['category', 'suspicious', 'turnover', 'transaction_count', 'io_ratio', 'age', 'nationality', 'is_pep', 'inactive_days_average', 'inactive_days_max', 'n_of_accounts', 'distinct_counterparties', 'channel_risk', 'atm_withdrawal', 'atm_deposit']


In [30]:
len(train[train['suspicious']==1])

9087

In [68]:
def label_age(row):
    if 30 <= row['age'] <= 50:
        return 1
    return 0

def label_nationality(row):
    if row['nationality'] in [141, 134, 123]:
        return 1
    return 0

train['age_new'] = train.apply(lambda row: label_age(row),axis=1)
train['nationality_new'] = train.apply(lambda row: label_nationality(row),axis=1)

In [75]:
indices_sus = []
indices_non_sus = []
suspicions = list(train['suspicious'])
customers = list(train.index)
for i in range(0, len(suspicions)):
    if suspicions[i] == 1:
        indices_sus.append(customers[i])
    else:
        indices_non_sus.append(customers[i])
        
train_sus = train.ix[indices_sus]
train_non_sus = train.ix[indices_non_sus].sample(9087)

In [76]:
train_non_sus.head()

,category,suspicious,turnover,transaction_count,io_ratio,age,nationality,is_pep,inactive_days_average,inactive_days_max,n_of_accounts,distinct_counterparties,channel_risk,atm_withdrawal,atm_deposit,age_new,nationality_new
customer,,,,,,,,,,,,,,,,,
90378226,0,0,14025.82,1.0,0.000000,53.0,94,0,69.74,140.0,1,1.0,0.416491,14025.82,240.616994,0,0
90786705,0,0,59.20,1.0,1.000000,25.0,33,0,122.70,123.0,3,1.0,0.067732,11.92,0.000000,0,0
90657248,0,0,20405.73,2.0,0.000000,36.0,191,0,26.36,53.0,1,1.0,0.233284,0.00,9.514731,1,0
90120852,0,0,10104.68,13.0,0.461538,18.0,33,0,9.69,97.0,2,1.0,0.081019,1659.88,5.347622,0,0
90811732,0,0,514216.16,2.0,0.000000,80.0,45,0,40.86,123.0,3,1.0,0.614039,0.00,293.757024,0,0


In [77]:
bigdata = train_sus.append(train_non_sus, ignore_index=True)

In [16]:
test = pd.read_csv(data_path+'test.csv', header=0, sep=",", index_col=0, parse_dates=True)

In [78]:
y_train = list(bigdata['suspicious'])
feature_cols = ['category', 'turnover', 'transaction_count', 'age_new', 'nationality_new']
#feature_cols = ['category', 'turnover', 'transaction_count', 'io_ratio', 'age', 'nationality', 'is_pep', 'inactive_days_average', 'inactive_days_max', 'n_of_accounts', 'distinct_counterparties', 'channel_risk', 'atm_withdrawal', 'atm_deposit']
X_train = bigdata.loc[:, feature_cols].values

In [83]:
def label_age(row):
    if 30 <= row['age'] <= 50:
        return 1
    return 0

def label_nationality(row):
    if row['nationality'] in [141, 134, 123]:
        return 1
    return 0

test['age_new'] = test.apply(lambda row: label_age(row),axis=1)
test['nationality_new'] = test.apply(lambda row: label_nationality(row),axis=1)

In [84]:
X_test = test.loc[:, feature_cols].values

In [80]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=0)
clf.fit(X_train, y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=5, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=100, n_jobs=1,
            oob_score=False, random_state=0, verbose=0, warm_start=False)

In [81]:
print(clf.feature_importances_)

[ 0.10952303  0.53917423  0.09470009  0.15735192  0.09925074]


In [85]:
predicts = list(clf.predict(X_test))

In [86]:
print(predicts[:100])

[1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1]


In [87]:
customers_predictions = []
for i in range(0,len(predicts)):
    if predicts[i]==1:
        customers_predictions.append(customers[i])

In [88]:
len(customers_predictions)

238541

In [89]:
results = pd.DataFrame()
results['customer'] = customers_predictions
results.to_csv('/Users/MeryemMhamdi/Desktop/Lauzhack_creditSuisse/results/RandomForest_new_features.csv', index=False)